In [2]:
from common import CV_DATA_DIR

import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import tensorflow as tf

# Reduce display precision on numpy array
np.set_printoptions(precision=2)

# Suppress warnings
tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(0)

# Get GPU physical devices
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Regression

1) Load dataset
2) Split dataset into training, cross-validation and test sets
3) Apply feature scaling
4) Train models with different degree (use polynomial features)
5) Choose a model with the lowest cross-validation set MSE
6) Compute test set MSE using model with chosen degree

In [ ]:
# Load data from CSV
data1 = np.loadtxt(CV_DATA_DIR / "playground" / "data_w3_ex1.csv", delimiter=",")

# Split data into separate arrays
x1 = data1[:,0]
y1 = data1[:,1]

# Convert 1D arrays to 2D
x1 = np.expand_dims(x1, axis=1)
y1 = np.expand_dims(y1, axis=1)

print(f"The shape of the inputs is: {x1.shape}")
print(f"The shape of the targets is: {y1.shape}")

In [ ]:
# Get 60% of the dataset for training set (40% remains)
x1_train, _x, y1_train, _y = train_test_split(x1, y1, test_size=0.40, random_state=1)
# Split remaining 40% into two equal parts: cross-validation set and test set
x1_cv, x1_test, y1_cv, y1_test = train_test_split(_x, _y, test_size=0.50, random_state=1)

del _x, _y

print(f"The shape of the training set (input) is: {x1_train.shape}")
print(f"The shape of the training set (target) is: {y1_train.shape}\n")
print(f"The shape of the cross validation set (input) is: {x1_cv.shape}")
print(f"The shape of the cross validation set (target) is: {y1_cv.shape}\n")
print(f"The shape of the test set (input) is: {x1_test.shape}")
print(f"The shape of the test set (target) is: {y1_test.shape}")

In [ ]:
# Create standard scaler
scaler = StandardScaler()

# Perform feature scaling
# 1) compute the mean and standard deviation
# 2) compute z-scores of each value
x1_train_scaled = scaler.fit_transform(x1_train)
x1_cv_scaled = scaler.transform(x1_cv)

print(f"Computed mean of the training set: {scaler.mean_.squeeze():.2f}")
print(f"Computed standard deviation of the training set: {scaler.scale_.squeeze():.2f}")

In [ ]:
# Initialize lists to save the errors, models, and feature transforms
train_mses = []
cv_mses = []
models = []
polys = []
scalers = []

# Loop over 10 times. Each adding one more degree of polynomial higher than the last.
for degree in range(1,11):
    # Add polynomial features to the training set
    poly = PolynomialFeatures(degree, include_bias=False)
    x1_train_mapped = poly.fit_transform(x1_train)
    polys.append(poly)

    # Scale the training set
    scaler = StandardScaler()
    x1_train_mapped_scaled = scaler.fit_transform(x1_train_mapped)
    scalers.append(scaler)

    # Create and train the model
    model = LinearRegression()
    model.fit(x1_train_mapped_scaled, y1_train)
    models.append(model)

    # Compute the training MSE (Mean Squared Error)
    yhat = model.predict(x1_train_mapped_scaled)
    train_mse = mean_squared_error(y1_train, yhat) / 2
    train_mses.append(train_mse)

    # Add polynomial features and scale the cross validation set
    x1_cv_mapped = poly.transform(x1_cv)
    x1_cv_mapped_scaled = scaler.transform(x1_cv_mapped)

    # Compute the cross validation MSE
    yhat = model.predict(x1_cv_mapped_scaled)
    cv_mse = mean_squared_error(y1_cv, yhat) / 2
    cv_mses.append(cv_mse)


In [ ]:
# Get the model with the lowest MSE (add 1 because list indices start at 0)
# This also corresponds to the degree of the polynomial added
degree = np.argmin(cv_mses) + 1
print(f"Lowest CV MSE is found in the model with degree={degree}")

In [ ]:
# Add polynomial features to the test set
X_test_mapped = polys[degree-1].transform(x1_test)

# Scale the test set
X_test_mapped_scaled = scalers[degree-1].transform(X_test_mapped)

# Compute the test MSE
yhat = models[degree-1].predict(X_test_mapped_scaled)
test_mse = mean_squared_error(y1_test, yhat) / 2

print(f"Training MSE: {train_mses[degree-1]:.2f}")
print(f"Cross Validation MSE: {cv_mses[degree-1]:.2f}")
print(f"Test MSE: {test_mse:.2f}")

# Classification

1) Load dataset
2) Split dataset into training, cross-validation and test sets
3) Apply feature scaling
4) Train models with different structure
5) Choose a model with the lowest cross-validation set mis-classification
6) Compute test set MSE using model with chosen degree

In [ ]:
# Load data from CSV
data2 = np.loadtxt(CV_DATA_DIR / "playground" / "data_w3_ex2.csv", delimiter=",")

# Split the inputs and outputs into separate arrays
x2 = data2[:,:-1]
y2 = data2[:,-1]

# Convert 1D to 2D
y2 = np.expand_dims(y2, axis=1)

print(f"the shape of the inputs x is: {x2.shape}")
print(f"the shape of the targets y is: {y2.shape}")

In [ ]:
# Get 60% of the dataset for training set (40% remains)
x2_train, _x, y2_train, _y = train_test_split(x2, y2, test_size=0.40, random_state=1)
# Split remaining 40% into two equal parts: cross-validation set and test set
x2_cv, x2_test, y2_cv, y2_test = train_test_split(_x, _y, test_size=0.50, random_state=1)

# Delete temporary variables
del _x, _y

print(f"the shape of the training set (input) is: {x2_train.shape}")
print(f"the shape of the training set (target) is: {y2_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x2_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y2_cv.shape}\n")
print(f"the shape of the test set (input) is: {x2_test.shape}")
print(f"the shape of the test set (target) is: {y2_test.shape}")

In [ ]:
# Create standard scaler
scaler = StandardScaler()

# Perform feature scaling
# 1) compute the mean and standard deviation
# 2) compute z-scores of each value
x2_train_scaled = scaler.fit_transform(x2_train)
x2_cv_scaled = scaler.transform(x2_cv)
x2_test_scaled = scaler.transform(x2_test)

In [ ]:
from keras.api.models import Sequential
from keras.api.layers import Dense

mode1 = Sequential([
    Dense(25, activation = 'relu'),
    Dense(15, activation = 'relu'),
    Dense(1, activation = 'linear')
], name="Model1")

model2 = Sequential([
    Dense(20, activation = 'relu'),
    Dense(12, activation = 'relu'),
    Dense(12, activation = 'relu'),
    Dense(20, activation = 'relu'),
    Dense(1, activation = 'linear')
], name="Model2")

model3 = Sequential([
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dense(8, activation = 'relu'),
    Dense(4, activation = 'relu'),
    Dense(12, activation = 'relu'),
    Dense(1, activation = 'linear')
], name="Model3")

models = (mode1, model2, model3)

In [ ]:
from keras.api.losses import BinaryCrossentropy
from keras.api.optimizers import Adam

nn_train_error = []
nn_cv_error = []

for model in models:
    # Specify the loss and optimizer
    model.compile(
        loss=BinaryCrossentropy(from_logits=True),
        optimizer=Adam(learning_rate=0.01),
    )
    # Train the model
    model.fit(
        x2_train_scaled, y2_train,
        epochs=200,
        verbose=0
    )

   # Get predictions for training set
    yhat = model.predict(x2_train_scaled)
    # Convert logits to probabilities
    yhat = tf.math.sigmoid(yhat)
    # Apply threshold to probabilities
    yhat = np.where(yhat >= 0.5, 1, 0)
    # Count mis-classifications
    train_error = np.mean(yhat != y2_train)
    nn_train_error.append(train_error)

    # Get predictions for CV set
    yhat = model.predict(x2_cv_scaled)
    # Convert logits to probabilities
    yhat = tf.math.sigmoid(yhat)
    # Apply threshold to probabilities
    yhat = np.where(yhat >= 0.5, 1, 0)
    # Count mis-classifications
    cv_error = np.mean(yhat != y2_cv)
    nn_cv_error.append(cv_error)

In [25]:
# Print the result
for model_num in range(len(nn_train_error)):
    print(f"Model {model_num+1}:")
    print(f"...Training Set Classification Error: {nn_train_error[model_num]:.5f}")
    print(f"...CV Set Classification Error: {nn_cv_error[model_num]:.5f}")

Model 1:
...Training Set Classification Error: 0.03333
...CV Set Classification Error: 0.15000
Model 2:
...Training Set Classification Error: 0.05000
...CV Set Classification Error: 0.17500
Model 3:
...Training Set Classification Error: 0.11667
...CV Set Classification Error: 0.20000


In [26]:
model_num = np.argmin(nn_cv_error)

# Get predictions for test set
yhat = models[model_num-1].predict(x2_test_scaled)
# Convert logits to probabilities
yhat = tf.math.sigmoid(yhat)
# Apply threshold to probabilities
yhat = np.where(yhat >= 0.5, 1, 0)
# Count mis-classifications
nn_test_error = np.mean(yhat != y2_test)

print(f"Selected Model: {model_num}")
print(f"Training Set Classification Error: {nn_train_error[model_num-1]:.4f}")
print(f"CV Set Classification Error: {nn_cv_error[model_num-1]:.4f}")
print(f"Test Set Classification Error: {nn_test_error:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Selected Model: 0
Training Set Classification Error: 0.1167
CV Set Classification Error: 0.2000
Test Set Classification Error: 0.1750
